#  Data Acquisition

## 1. Setup

In [ ]:
import logging
! pip install numpy==1.21
logging.basicConfig(
    format="%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s",
    datefmt="%H:%M",
    level=logging.INFO,
)

for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)
from madminer import MadMiner

## 2.Determine the form of the Lagrangian

In [ ]:
miner = MadMiner()

miner.add_parameter(
    lha_block="SMEFT",
    lha_id=118,
    parameter_name="cHq3Re33",
    morphing_max_power=2,
    param_card_transform="1000*theta",
    parameter_range=(-1.0, 1.0),
)
miner.add_parameter(
    lha_block="SMEFT",
    lha_id=136,
    parameter_name="cHuRe33",
    morphing_max_power=2,
    param_card_transform="1000*theta",
    parameter_range=(-1.0, 1.0),
)

miner.add_benchmark({"cHq3Re33": 0.0, "cHuRe33": 0.0}, 'sm')

miner.set_morphing(include_existing_benchmarks=True, max_overall_power=2)

miner.save("data/setup_em.h5")

## 3.Run Madgraph and Pythia8

In [ ]:
import os

from madminer.core import MadMiner
from madminer.delphes import DelphesReader

## 4.Run Delphes

In [ ]:
delphes = DelphesReader("data/setup_please.h5")
j = 1
for m in ["sm"]:
    delphes.add_sample(
        lhe_filename=f'mg_processes/{fileName}/Events/run_0{j}/unweighted_events.lhe.gz',
        hepmc_filename=f'mg_processes/{fileName}/Events/run_0{j}/tag_1_pythia8_events.hepmc.gz',
        sampled_from_benchmark=m,

        is_background=False,
    )
    j += 1
delphes.run_delphes(
    delphes_directory=mg_dir + '/Delphes',
    delphes_card='cards/delphes_card.dat',
    log_file='logs/delphes.log',
)

## 5.Choose Observables

In [ ]:
delphes.add_default_observables(n_leptons_max=2,n_photons_max=0,n_jets_max=4) #todo
delphes.analyse_delphes_samples()
delphes.save("data/delphes_0511_2xiao.h5")